0. Import file

In [1]:
from instance_pyfile import *
from support_pyfile import *
import time


[[0.0, 61.07372593840988, 74.09453421137081, 59.81638571495272, 51.07837115648854, 24.413111231467404, 81.9084855189009, 18.439088914585774, 55.57877292636101, 66.76076692189807, 56.08029957123981, 48.25971404805462, 63.702433234531945, 66.61080993352356, 92.65527507918802, 52.630789467763066], [61.07372593840988, 0.0, 16.0, 12.806248474865697, 28.160255680657446, 36.796738985948195, 34.0147027033899, 45.45327270945405, 42.15447781671598, 26.248809496813376, 20.12461179749811, 34.0147027033899, 4.0, 35.510561809129406, 32.7566787083184, 36.05551275463989], [74.09453421137081, 16.0, 0.0, 27.202941017470888, 43.73785545725808, 49.73932046178355, 20.518284528683193, 57.0087712549569, 57.28001396647874, 20.024984394500787, 35.17101079013795, 37.16180835212409, 16.492422502470642, 30.14962686336267, 28.0178514522438, 51.419840528729765], [59.81638571495272, 12.806248474865697, 27.202941017470888, 0.0, 17.0, 37.12142238654117, 46.52956049652737, 47.01063709417264, 30.083217912982647, 38.8973

1. Confige LLM

In [2]:
import google.generativeai as genai

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE"
    }, 
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE"
    },
]

genai.configure(api_key='YOUR_API_KEY')

model = genai.GenerativeModel(model_name='gemini-pro', safety_settings=safety_settings)

config = genai.GenerationConfig(temperature=1.0)


In [3]:
best_cur = 999999999
best_now = 999999990
cur_temp = 1.0

2. Sample prompt constructions in paper
- task_description: about TSP
- in_context: instances from the population.
- task_instruction: step-by-step instructions for LLM to follow

In [4]:
description = 'You are given a list of points with coordinates: {' + list2str() + '}. Your task is to find a trace, with the shortest possible length, that trverses each point exactly once.\n'

in_context = 'Below are some previous traces and their lengths. The traces are arranged in descending order based on their lengths, where lower values are better.\n'
#in_context += pool2examples(Pool)                  ! NOTICE that each generation, Pool change.

task_instruction = 'Please follow the instruction step-by-step to generate new trace:\n'
task_instruction += 'Step 1. Select two traces from the above traces.\n'
task_instruction += 'Step 2. Crossover the two traces chosen in Step 1 and generate a new trace.\n'
task_instruction += 'Step 3. Mutate the trace generated in Step 2 and generate a new trace.\n'
task_instruction += ('Step 4. Keep the generated trace generated in Step 3, repeat Step 1, 2, 3, until you have ' + str(N) + ' generated traces.\n')
task_instruction += 'Directly give me all the chosen traces at Step 1, bracketed them with <selection> and </selection>, and all the generated traces at Step 3, bracketed them with <res> and </res>. Not any explanation needed.'

: 

2.5. Using workbook to write data to excel
- Each generation, write to file some information: #gen, num_calls, best_found, worst_found, average
- need to create workbook and newsheet at once then avoid overwritting.
- each tsp_instances need different workbook and worksheet.

In [4]:
import xlsxwriter

workbook = xlsxwriter.Workbook("trcd_rue_15_3.xlsx")

worksheet = workbook.add_worksheet("rue_15_tsp")

column_labels = ['#gen', 'num_calls', 'best_found', 'worst_found', 'average_sol']

for i in range(len(column_labels)) :
    worksheet.write(0, i, column_labels[i])


3. Algorithm1 (from paper):
- each API call, provide all N child in the population as in-context learn for LLM
- for 1 call, LLM try to generate N new offspring from the provided population.
- Note with the Self-Adaption

In [5]:
Pool = randomFirstN(n, N)

g = 1

def proc() :
    global Pool, g
    global workbook, worksheet
    num_calls = 0
    
    while g <= G :

        if g % 50 == 0:
            print("break after each 50 generations! ")
            time.sleep(600)

        num_calls = 0

        start_gene = time.time()

        prompt = description + (in_context + pool2examples(Pool)) + task_instruction

        listOff = []
        best_cur = Pool[N-1].length

        while len(listOff) < N :

            config = genai.GenerationConfig(temperature=cur_temp)

            start_call = time.time()

            
            response = model.generate_content(contents=prompt, generation_config=config)

            num_calls += 1

            spent_call = time.time() - start_call

            print(f"Time for the API calls in generation {g}: {spent_call}")

            newGen = cutGenTrace(response.text)
            for s in newGen:
                if checkPermu(s, n) :
                    listOff.append(s)
        #P_sharp = transform(listOff[:N])

        Pool = updatePool(Pool, listOff[:N], N)
    
        best_now = Pool[N-1].length

        if check_stuck(best_cur, best_now, K=10):
            cur_temp += 0.1
            if cur_temp == 2.0 :
                cur_temp = 1.5
            
            print("Self-Adaptation!")

        spent_gene = time.time() - start_gene
        print(f"Total time for the generation {g}: {spent_gene}")
        print(f"best solution now is: {best_now}")
        print(f"worst solution now is: {Pool[0].length}")

        worksheet.write(g, 0, g)
        worksheet.write(g, 1, num_calls)
        worksheet.write(g, 2, best_now)
        worksheet.write(g, 3, Pool[0].length)
        worksheet.write(g, 4, )
        g += 1

try:
    proc()
except:
    if g < G:
        proc()
        
print(Pool[N-1])

Time for the API calls in generation 1: 20.397091150283813
